<a href="https://colab.research.google.com/github/rubuntu/Taller_Introduccion_a_Ciencia_de_Datos_IA_e_Ingenieria_de_Datos/blob/main/sesion_16_fundamentos_de_rag_con_hugging_face_chromadb.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

# 📘 Sesión 16 – Fundamentos de RAG con Hugging Face + ChromaDB

## Objetivos

* Comprender los conceptos básicos de RAG y su arquitectura.
* Aprender a transformar texto → embeddings → almacenar en base vectorial.
* Construir un mini-RAG con ChromaDB y Hugging Face embeddings.

---

## Contenido

1. **Concepto de RAG**:

   * Separar *memoria a largo plazo* (vector store) del LLM.
   * Flujo: **Pregunta → Embedding → Recuperación → Contexto → LLM → Respuesta**.

2. **Primer prototipo**:

   * Usar `sentence-transformers` de Hugging Face para embeddings.
   * Guardar y consultar embeddings en **ChromaDB**.
   * Pasar contexto recuperado a un LLM (ej. `transformers` o `openai`).




---



## 1. ¿Qué es RAG?

* **RAG** = *Retrieval-Augmented Generation*.
* Es una técnica que combina dos mundos:

  1. **Recuperación de información (IR / search engine)** → buscar documentos relevantes en una base de datos.
  2. **Generación con modelos de lenguaje (LLM)** → producir una respuesta en lenguaje natural.

👉 La idea principal: **el modelo no depende solo de lo que tiene en sus pesos, sino que se “alimenta” de fuentes externas actualizadas**.

Ejemplo:

> Preguntas: *“¿Quién es el actual presidente de Paraguay?”*
>
> * Un LLM puro (sin RAG) puede fallar si no está actualizado.
> * Un RAG consulta primero en su base/vector store con datos recientes → encuentra el documento correcto → luego el LLM genera la respuesta basándose en ese contexto.

---

## 2. Arquitectura básica de RAG

Se suele dividir en **4 pasos principales**:

### 🔹 Paso 1. **Indexación (offline)**

* Se prepara una colección de documentos (ej: artículos de Wikipedia, PDFs, noticias, etc.).
* Se dividen en **chunks** (trozos) para que no sean demasiado largos.
* Cada chunk se convierte en un **embedding vectorial** (ej: usando *SentenceTransformers*).
* Los embeddings se guardan en un **vector store** (ChromaDB, Pinecone, Weaviate, FAISS, etc.) junto con el texto original.

### 🔹 Paso 2. **Consulta del usuario**

* El usuario hace una pregunta en lenguaje natural.
* Esa pregunta también se convierte en un **embedding**.

### 🔹 Paso 3. **Recuperación**

* Se busca en el vector store los **chunks más similares** al embedding de la pregunta.
* Estos fragmentos relevantes forman el **contexto** que se pasará al modelo.

### 🔹 Paso 4. **Generación**

* Se construye un **prompt** que combina:

  * La pregunta del usuario.
  * Los documentos recuperados como *contexto*.
* Se envía este prompt al **LLM** (Gemma, Mistral, Llama, Qwen, etc.).
* El modelo genera la respuesta final, apoyándose en el contexto.

---

## 3. Diagrama de flujo simplificado

```
Usuario → Pregunta
   ↓
Embeddings de la pregunta
   ↓
Vector Store (ChromaDB) → Recupera chunks relevantes
   ↓
Construcción del Prompt (Contexto + Pregunta)
   ↓
LLM (ej: Gemma-3n) → Genera respuesta
   ↓
Respuesta final al usuario
```

---

## 4. Beneficios de RAG

* **Actualización dinámica:** basta con cambiar la base de datos, no hay que reentrenar el LLM.
* **Precisión y grounding:** el modelo se apoya en fuentes externas verificables.
* **Eficiencia:** usar un LLM relativamente pequeño + base vectorial puede superar a un LLM gigante sin RAG.
* **Menor alucinación:** como el modelo tiene el contexto correcto, es menos probable que invente información.

---

## 5. Limitaciones y desafíos

* **Calidad de la base:** si la base de documentos es pobre, el LLM no podrá responder bien.
* **Chunking:** si se cortan mal los textos, puede faltar contexto.
* **Ventana de contexto:** los LLM tienen límite en la cantidad de tokens que pueden recibir.
* **Coste de recuperación:** búsquedas vectoriales en bases muy grandes requieren optimización.

---

✅ **En resumen:**
RAG es una arquitectura que **amplía la memoria de los LLM** conectándolos a una base de conocimiento externa. Esto permite respuestas más precisas, actualizadas y con respaldo en datos, en lugar de depender solo de lo aprendido durante el entrenamiento del modelo.

---

## Demo Código

In [ ]:
# ========================
# Instalación de librerías
# ========================
%%capture
!pip install -q -U wikipedia-api chromadb bitsandbytes gradio


In [ ]:
# ========================
# 1. Cargar datos de Wikipedia (Paraguay)
# ========================
import wikipediaapi

wiki_wiki = wikipediaapi.Wikipedia(
    language='es',
    user_agent='MiProyectoRAG/1.0 (https://github.com/rubuntu)'
)

paginas = [
    "Paraguay",
    "Geografía de Paraguay",
    "Economía de Paraguay",
    "Cultura de Paraguay",
    "Historia de Paraguay",
    "Asunción",
    "Guerra de la Triple Alianza",
    "Guerra del Chaco",
]

docs = []
for titulo in paginas:
    page = wiki_wiki.page(titulo)
    if page.exists():
        docs.append(page.text)
        print(f"Cargado: {titulo}")
print(f"Total documentos cargados: {len(docs)}")


In [ ]:
# ========================
# 2. Chunking para RAG
# ========================
def chunk_text(text, max_chars=1200, overlap=200):
    chunks, start = [], 0
    while start < len(text):
        end = start + max_chars
        chunk = text[start:end]
        chunks.append(chunk)
        start += max_chars - overlap
    return chunks

all_chunks = []
for doc in docs:
    all_chunks.extend(chunk_text(doc))

print(f"Total chunks creados: {len(all_chunks)}")


Explicación:

* **Objetivo:** dividir documentos largos en fragmentos (*chunks*) más pequeños para que los embeddings y el LLM los procesen mejor en RAG.
* **Parámetros:**

  * `max_chars=1200` → tamaño máximo de cada chunk en caracteres.
  * `overlap=200` → solapamiento entre chunks para no perder continuidad.
* **Funcionamiento:** avanza por el texto tomando bloques de 1200 caracteres, retrocede 200 y sigue → así asegura que frases no queden cortadas.
* **Resultado:** `all_chunks` contiene todos los fragmentos de todos los documentos, listos para generar embeddings e indexar en ChromaDB.

👉 En pocas palabras: **corta los documentos grandes en trozos solapados, más fáciles de buscar y recuperar en el RAG.**


In [ ]:
# ========================
# 3. Embeddings y ChromaDB
# ========================
from sentence_transformers import SentenceTransformer
import chromadb

emb_model = SentenceTransformer("all-MiniLM-L6-v2")
embeddings = emb_model.encode(all_chunks, batch_size=16, show_progress_bar=True)

client = chromadb.Client()
collection = client.create_collection("paraguay_wiki")

for i, chunk in enumerate(all_chunks):
    collection.add(documents=[chunk], embeddings=[embeddings[i].tolist()], ids=[str(i)])

print("✅ Base vectorial creada con ChromaDB.")


Explicación:

* Se carga un **modelo de embeddings** (`all-MiniLM-L6-v2`) para convertir texto en vectores numéricos que capturan su significado.
* Se generan embeddings para todos los *chunks* de texto (`emb_model.encode`).
* Se crea una **colección en ChromaDB** llamada `"paraguay_wiki"`.
* Cada chunk se guarda en la colección junto con su embedding y un ID único.
* Resultado: ya tienes una **base vectorial** lista para buscar y recuperar fragmentos relevantes en un flujo RAG.

👉 En pocas palabras: **convierte los chunks en vectores y los indexa en ChromaDB para poder hacer búsquedas semánticas.**


In [ ]:
# ========================
# 4. Cargar Modelo
# ========================
from transformers import AutoTokenizer, AutoModelForCausalLM

model_id = "unsloth/gemma-3n-E2B-it-unsloth-bnb-4bit"

tokenizer = AutoTokenizer.from_pretrained(model_id, trust_remote_code=True, use_fast=False)
llm = AutoModelForCausalLM.from_pretrained(
    model_id,
    device_map="auto",
    trust_remote_code=True
)

print("✅ Modelo cargado:", model_id)


Explicación:

* Se usa **Hugging Face Transformers** para cargar un **tokenizer** y un **modelo de lenguaje causal (LLM)**.
* `model_id` apunta a **`unsloth/gemma-3n-E2B-it-unsloth-bnb-4bit`**, una versión de **Gemma-3n** (variante **E2B**) ya **quantizada en 4 bits con Unsloth**, lo que la hace más ligera.
* El **tokenizer** convierte texto en tokens (IDs numéricos).
* El **modelo LLM** (cargado con `AutoModelForCausalLM`) genera texto a partir de esos tokens.
* `device_map="auto"` envía automáticamente el modelo a GPU si está disponible.

👉 En pocas palabras: **se carga Gemma-3n (E2B, quantizado 4-bit) con Unsloth, optimizado para bajo consumo de memoria y buen rendimiento en tareas de chat/instrucción.**




In [ ]:
# ========================
# 5. Función RAG
# ========================
def to_gemma_chat(user_text):
    return f"<bos><start_of_turn>user\n{user_text}<end_of_turn>\n<start_of_turn>model\n"

def rag_answer(question, top_k=3, max_new_tokens=200, show_context=False):
    # Recuperar contexto
    query_emb = emb_model.encode([question])
    results = collection.query(query_embeddings=query_emb.tolist(), n_results=top_k)
    context = "\n".join(results["documents"][0])

    if show_context:
        print("\n--- CONTEXTO ---")
        print(context[:1000] + ("..." if len(context) > 1000 else ""))
        print("--- FIN CONTEXTO ---\n")

    # Prompt
    prompt = f"""
Responde a la pregunta basándote únicamente en el siguiente contexto.
No incluir la palabra "Contexto" en la respuesta.
Si el contexto no contiene la respuesta, responde "No lo sé".

Contexto:
{context}

Pregunta: {question}

Respuesta:
"""
    chat_prompt = to_gemma_chat(prompt)

    inputs = tokenizer(chat_prompt, return_tensors="pt", truncation=True, max_length=4096).to("cuda")
    outputs = llm.generate(
        **inputs,
        max_new_tokens=max_new_tokens,
        do_sample=True,
        temperature=0.3,
        top_p=0.9,
        pad_token_id=tokenizer.eos_token_id
    )
    decoded = tokenizer.decode(outputs[0], skip_special_tokens=True)

    # --- limpieza de la salida ---
    # cortar donde empieza <start_of_turn>model
    if "<start_of_turn>model" in decoded:
        answer = decoded.split("<start_of_turn>model")[-1].strip()
    else:
        # fallback: quitar el prompt entero si aparece
        answer = decoded.replace(chat_prompt, "").strip()

    return answer




---

Explicación

1. **to\_gemma\_chat**

   * Adapta el texto del *prompt* al formato especial que entiende **Gemma** (`<bos><start_of_turn>user ... <start_of_turn>model`).

2. **rag\_answer**

   * **Recupera contexto:** convierte la pregunta a embedding y busca en ChromaDB los chunks más similares (`top_k`).
   * **Construye el prompt:** inserta el contexto + la pregunta en una instrucción clara (“responde solo con el contexto o di ‘No lo sé’”).
   * **Prepara el chat\_prompt:** lo transforma al formato de Gemma usando `to_gemma_chat`.
   * **Genera respuesta:** pasa el prompt al modelo (`llm.generate`), limitando longitud y usando sampling controlado (`temperature`, `top_p`).
   * **Limpieza:** elimina el eco del prompt y se queda solo con la respuesta del modelo.
   * **Devuelve la respuesta final.**

---

👉 Resumen: (Pregunta → Embedding → Búsqueda en Chroma → Prompt → LLM → Respuesta)    
La función **RAG** toma una pregunta → busca los fragmentos de texto relevantes → arma un prompt con ese contexto → lo da al modelo (Gemma) → limpia la salida → devuelve la respuesta clara.

---


In [ ]:
# ========================
# 6. Preguntas de ejemplo
# ========================
questions = [
    "¿Cuál es la capital de Paraguay?",
    "¿Quién fue Francisco Solano López?",
    "¿Qué papel tuvo Paraguay en la Guerra de la Triple Alianza?",
    "¿Cuáles son los principales productos de exportación de Paraguay?"
]

for q in questions:
    print("\n❓", q)
    print("💡", rag_answer(q, ))

In [ ]:
# ========================
# 7. Interfaz con Gradio
# ========================
import gradio as gr

def ask_question(query):
    return rag_answer(query)

demo = gr.Interface(
    fn=ask_question,
    inputs=gr.Textbox(lines=2, placeholder="Escribe tu pregunta sobre Paraguay..."),
    outputs="text",
    title="RAG sobre Paraguay (Wikipedia + LLM)",
    description="Haz preguntas sobre Paraguay usando RAG con ChromaDB y un modelo abierto (Gemma)."
)

demo.launch(share=True)


---

# 📘 Preguntas de discusión sobre RAG

---

## 1. ¿Por qué un LLM necesita un vector store externo para RAG?

* **Limitación de memoria de los LLM:**
  Un LLM no puede "recordar" información fuera de lo que se entrenó. Si se quiere consultar datos **actualizados o muy específicos**, hay que tener una fuente externa.

* **Búsqueda eficiente:**
  El vector store convierte textos en **embeddings** (vectores numéricos). Eso permite hacer búsquedas semánticas rápidas en miles o millones de documentos.
  Ejemplo: la pregunta *“¿Cuál es la capital de Paraguay?”* se convierte en un vector → se compara contra todos los chunks → se recuperan los más relevantes.

* **Ventaja sobre embeddings directos:**
  Sin un vector store, tendrías que comparar la pregunta con todos los documentos cada vez (coste enorme). Con un motor especializado (ChromaDB, FAISS, Pinecone) esto se resuelve en milisegundos usando técnicas como *Approximate Nearest Neighbors (ANN)*.

👉 En resumen: el vector store es el **“cerebro externo”** que permite al LLM acceder a datos organizados y recuperarlos de forma rápida y precisa.

---

## 2. ¿Qué limitaciones tendría un mini-RAG con pocas docenas de documentos?

* **Cobertura limitada:**
  Con pocos documentos, la base de conocimiento es reducida → el modelo solo puede responder a preguntas relacionadas con ese corpus.

* **Alto riesgo de “No sé” o alucinaciones:**
  Si la información buscada no está en esos documentos, el modelo puede inventar respuestas o decir que no sabe.

* **Menor diversidad semántica:**
  Un vector store pequeño no ofrece gran variedad de embeddings → recupera siempre los mismos textos aunque no sean perfectos.

* **Buen caso de uso:**
  Es útil como **demo educativo o POC (proof of concept)**, pero insuficiente para producción donde se requieren millones de artículos, actualizaciones periódicas y calidad robusta.

---

## 3. ¿Cómo escalarías esto a millones de documentos?

Para llevar un RAG de un prototipo a producción a gran escala, habría que:

* **Indexación distribuida:**

  * Usar bases vectoriales diseñadas para grandes volúmenes (Pinecone, Weaviate, Milvus, Vespa, FAISS con sharding).
  * Permitir búsqueda paralela en varios nodos.

* **Preprocesamiento avanzado:**

  * Buen chunking dinámico (según párrafos, secciones, etc.).
  * Enriquecer con metadatos (fecha, fuente, tipo de documento) para filtrar más rápido.

* **Técnicas de búsqueda híbrida:**

  * Combinar **búsqueda semántica** (embeddings) con **búsqueda lexical** (BM25, keyword search).
  * Esto reduce falsos positivos y mejora la relevancia.

* **Optimización de embeddings:**

  * Usar modelos más robustos y multilingües.
  * Batch processing para indexar millones de textos sin colapsar memoria.

* **Pipeline escalable:**

  * Arquitectura de microservicios: un servicio para embeddings, otro para indexación, otro para retrieval.
  * Cachear consultas frecuentes.
  * Actualizar la base vectorial de manera incremental (streaming ingestion).

👉 En pocas palabras: pasar de un mini-RAG a millones de documentos requiere **infraestructura especializada, almacenamiento distribuido y técnicas de búsqueda híbrida para mantener velocidad y precisión.**

---

